In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt     
import numpy as np
from sklearn.preprocessing import StandardScaler
import cv2

In [ ]:
# Calculate z-scores for x and y columns
# make this code into a function that takes a dataframe and returns the cleaned dataframe
        
from scipy.stats import zscore
def clean_outliers(df):
    z_scores_x = zscore(df.iloc[:,0])
    z_scores_y = zscore(df.iloc[:,1])

    # Define threshold for z-score (e.g., 3)
    threshold = 3

    # Identify outliers based on z-scores
    outliers_x = df[abs(z_scores_x) > threshold]
    outliers_y = df[abs(z_scores_y) > threshold]

    # Remove outliers from DataFrame
    cleaned_df = df[~df.index.isin(outliers_x.index) & ~df.index.isin(outliers_y.index)]
    return cleaned_df

In [ ]:
# lets decide what it means for contact to happen 
close_prox = 100pixels 
distant_prox = 500pixels 

In [ ]:
# draw lines on the frame at each 50 pixels 
# the resolution is 768 x 576
for i in range(0, 768, 50):
    cv2.line(frame, (i, 0), (i, 576), (0, 255, 0), 1)

In [ ]:
from moviepy.editor import VideoFileClip
video_folder = '/Users/andrei-macpro/Documents/Data/videos/meal_videos' 

durations_videos = []
file_names = []

for folder_name in sorted(os.listdir(video_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(video_folder, folder_name)
    if os.path.isfile(file_path):
        clip = VideoFileClip(file_path)
        durations_videos.append(clip.duration)
        file_names.append(folder_name.split('.')[0])

durations_videos = pd.DataFrame({'file_name': file_names, 'duration': durations_videos})

In [ ]:
tracking_folder = '/Users/andrei-macpro/Documents/Data/openpose/meal/tracking/meal_tracking_2' 

In [ ]:
file_path = '/Users/andrei-macpro/Documents/Data/openpose/play/tracking/tracking/1047_play'
tracks = {}
durations = []
for file in sorted(os.listdir(file_path)):
        # load the file in a pandas dataframe
    if file == ".DS_Store":
        continue    
    filepath = os.path.join(file_path, file)
    df = pd.read_csv(filepath, index_col=0)
    df = df[~df.index.duplicated(keep='first')]
    df.columns = ['x_' + file.split('.')[0], 'y_' + file.split('.')[0]]
    df = clean_outliers(df)
    tracks[file.split('.')[0]] = df



combined_data = pd.concat([df for df in tracks.values()], axis=1)
track_names = list(tracks.keys())
for i in range(0, len(track_names), 2):
    # Get the coordinates of the two subjects
    x1 = combined_data['x_' + track_names[i]]
    y1 = combined_data['y_' + track_names[i]]
    x2 = combined_data['x_' + track_names[i+1]]
    y2 = combined_data['y_' + track_names[i+1]]

# Calculate the Euclidean distance between the two subjects
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    print(distance)

    # Find the indices where the distance is less than 100
    close_indices = np.where(distance < 100)[0]
    if len(close_indices) == 0:
        continue
    start = close_indices[0]

        # Iterate over the close indices
    for i in range(1, len(close_indices)):
    # If the current index is not consecutive to the previous one,
    # it means a new contact period has started
        if close_indices[i] != close_indices[i-1] + 1:
        # The duration of the contact period is the difference between the start and end times
            duration = close_indices[i-1] - start
            durations.append(duration)

        # Update the start of the new contact period
            start = close_indices[i]
# Count how many times the distance is less than 100
    duration = close_indices[-1] - start
    durations.append(duration)

print(f"The durations of the contact periods were: {durations}")
print(f"The two subjects were closer than 100 a total of {len(durations)} times.")

In [ ]:
# average close contact duration
# frequency of close contact


In [ ]:
tracking_folder = '/Users/andrei-macpro/Documents/Data/openpose/meal/tracking/meal_tracking_2' 

stats = {}
for folder_name in sorted(os.listdir(tracking_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(tracking_folder, folder_name)
    tracks = {}
    durations = []
    for file in sorted(os.listdir(file_path)):
            # load the file in a pandas dataframe
        if file == ".DS_Store":
            continue    
        filepath = os.path.join(tracking_folder, file_path, file)
        df = pd.read_csv(filepath, index_col=0)
        df = df[~df.index.duplicated(keep='first')]
        df.columns = ['x_' + file.split('.')[0], 'y_' + file.split('.')[0]]
        df = clean_outliers(df)
        tracks[file.split('.')[0]] = df



    combined_data = pd.concat([df for df in tracks.values()], axis=1)
    track_names = list(tracks.keys())
    for i in range(0, len(track_names), 2):
        # Get the coordinates of the two subjects
        x1 = combined_data['x_' + track_names[i]]
        y1 = combined_data['y_' + track_names[i]]
        x2 = combined_data['x_' + track_names[i+1]]
        y2 = combined_data['y_' + track_names[i+1]]

    # Calculate the Euclidean distance between the two subjects
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        # Find the indices where the distance is less than 100
        close_indices = np.where(distance < 100)[0]
        if len(close_indices) == 0:
            continue
        start = close_indices[0]

            # Iterate over the close indices
        for i in range(1, len(close_indices)):
        # If the current index is not consecutive to the previous one,
        # it means a new contact period has started
            if close_indices[i] != close_indices[i-1] + 1:
            # The duration of the contact period is the difference between the start and end times
                duration = close_indices[i-1] - start
                durations.append(duration)

            # Update the start of the new contact period
                start = close_indices[i]
    # Count how many times the distance is less than 100
        duration = close_indices[-1] - start
        durations.append(duration)

    duration_video = durations_videos.loc[durations_videos['file_name'] == folder_name]['duration'].values[0]
    if folder_name not in stats:
        stats[folder_name] = {}
    stats[folder_name] = {'average_contact_duration': np.mean(durations)/duration_video*100
                                          , 'frequency_contact': len(durations)/duration_video*100}




In [ ]:
stats

In [ ]:
# turn stats into a pandas dataframe
stats_df = pd.DataFrame(stats).T
stats_df

In [ ]:
# replace nan values with 0 in stats_df
stats_df = stats_df.fillna(0)
stats_df

In [ ]:
stats_df.to_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/contact_frequency_duration.csv')